<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Newton_Method_for_Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing required libraries
!pip install torchinfo
import torchinfo
import torch
from torch import nn,optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"

In [3]:
# Creating our own LeNet5

class LeNet5(nn.Module):
  def __init__(self):
    super(LeNet5,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5) # in channel , out channe, kernel
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d((2,2))

    self.conv2 = nn.Conv2d(6,16,5)  # in channel , out channe, kernel
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d((2,2))

    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    # x = F.max_pool2d(F.relu(self.conv1(x)),(2, 2))
    # x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = self.conv1(x)
    x = self.relu1(x)
    x= self.maxpool1(x)

    x = self.conv2(x)
    x = self.relu2(x)
    x= self.maxpool2(x)

    x = x.view(-1,int(x.nelement() / x.shape[0]))

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)



    return x



model = LeNet5().to(device)

In [4]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
train_transforms = transforms.Compose([ transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])
train_data = CIFAR10(root="./train/", train=True, download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader( train_data,batch_size=16, shuffle=True)

100%|██████████| 170498071/170498071 [00:04<00:00, 42386773.88it/s]


Extracting ./train/cifar-10-python.tar.gz to ./train/


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.01)


In [ ]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):
  epoch_loss = 0.0
  for inputs, labels in trainloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    def closure():
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      global epoch_loss
      epoch_loss += loss.item()
      return loss
    optimizer.step(closure)
  print("Epoch: {} Loss: {}".format(epoch,epoch_loss/len(trainloader)))



Epoch: 0 Loss: nan
